### Library used and deeclare data needed

In [2]:
import numpy as np
import csv
import pandas as pd
from numpy.typing import NDArray
from scipy.stats import zscore
from pytictoc import TicToc
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold, cross_val_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
from sklearn.metrics import accuracy_score

In [4]:
kernel_fcn = 'rbf' # is this gaussian?
opts_csv_fold = 5
with open('./data/algolabels.csv', newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            algo_labels = row

y = np.loadtxt('./data/ybin.csv', delimiter=',', skiprows=1)
z = pd.read_csv('./data/z_M.csv', header=None, dtype=np.float64)

ninst, nalgos = y.shape
w = np.ones((ninst, nalgos))

""" Problem 1 - Normalization generate different result with MATLAB """
# TODO: Lam: review
# Matlab Mean: -2.97455980182589e-16,-1.42443708819831e-16
# Matlab SD: sigma: 1.42766287722398,1.88883050190813
z_norm = (z-np.mean(z, axis=0))/np.std(z, ddof=1, axis=0)
print(np.std(z, ddof=1, axis=0))
pd.DataFrame(z_norm).to_csv('z_f.csv', header=None, index=None)
# scaler = StandardScaler().fit(z)

cvcmat = np.zeros((nalgos, 4))


0   -4.336154e-16
1    2.178551e-16
dtype: float64
hahah
0    1.427663
1    1.888831
dtype: float64


### Check Consistency

In [ ]:
z_norm_M = pd.read_csv('./data/z_norm_M.csv', header=None, dtype=np.float64)
z_norm_P = pd.read_csv('z_f.csv', header=None, dtype=np.float64)

tolerance = 1e-10
are_close = np.isclose(z_norm_M.values, z_norm_P.values, atol=tolerance)
results = are_close.all()

print("Are all elements close within the tolerance level: ", results)
if not results:
    mismatches = np.where(~are_close)
    print("Mismatch found at positions:", mismatches)

### Training

In [ ]:
def fit_libsvm(z, y, kkv, kernel_given):
    accuracy= dict()
    for k, v in kkv.items():
        train_index, test_index = v[0], v[1]
        # prepare training data
        x_train = [z[i] for i in train_index]
        y_train = [y[i] for i in train_index]
        # prepare test data
        x_test = [z[i] for i in test_index]
        y_test = [y[i] for i in test_index]
        svm = SVC(kernel=kernel_given, C=1.0, random_state = 0)
        svm.fit(x_train, y_train)
        y_pred = svm.predict(x_test)
        # calculate accuracy
        accuracy[k] = accuracy_score(y_test, y_pred)
        
    return accuracy

In [ ]:
def fit_matsvm(z, y, w, skf, kernel_given, params):
    # TODO Set up parallel workers in pool
    

    # Scikit-learn lib need to ensuring data contiguity
    z = np.ascontiguousarray(z)
    y = np.ascontiguousarray(y)
    w = np.ascontiguousarray(w)
    
    # Check if hyperparameter is given by user
    if(np.isnan(params)):

        # Initialize a random number generator
        np.random.seed(0)

        # Retrieve default hyperparameters for fitcsvm and sets the range for the box constraint (C) and kernel scale
        # Define the range for C and gamma in a logarithmic scale
        param_grid = {
        'C': np.logspace(-10, 4, base=2, num=15),
        'gamma': np.logspace(-10, 4, base=2, num=15)
        }

        # By default, the class_weight=None represent equal weight
        svm_model = SVC(kernel=kernel_given, class_weight=None, random_state=0)

        # steps = list()
        # steps.append(('scaler', StandardScaler()))
        # steps.append(('model', svm_model))
        # pipeline = Pipeline(steps=steps)

        # Used for exhaustive search over specified parameter values for the SVM. The param_grid defines 
        # the range over which C and gamma will be tuned.
        # GridSearchCV for optimizing the hyperparameters
        # TODO Lam: Figure out a better method
        grid_search = GridSearchCV(
            svm_model, param_grid, 
            scoring='accuracy', # 'roc_auc'
            cv=skf, 
            verbose=0
            #, n_jobs=nworkers if nworkers != 0 else None,
            )
        
        # OPT1: 
        # Fit a probability calibration model with trained SVM
        # print(z.shape, y.shape, w.shape)
        # TODO Lam: no matter what c, gamma or kfold, but the best_svm should be same 
        grid_search.fit(z, y, sample_weight=w)   # Fit GridSearchCV
        best_svm = grid_search.best_estimator_
       
        calibrator = CalibratedClassifierCV(best_svm, cv='prefit', method='sigmoid')
        calibrator.fit(z, y, sample_weight=w)

        # OPT2: retrain
        # calibrator = CalibratedClassifierCV(best_svm, cv=skf, method='sigmoid')
        # calibrator.fit(z_norm, y, sample_weight=w)

        best_C = grid_search.best_params_['C']
        best_g = grid_search.best_params_['gamma']

        y_sub = calibrator.predict(z)
        p_sub = calibrator.predict_proba(z)

        # Making predictions on the same data to simulate resubstitution prediction
        y_hat = y_sub
        p_hat = p_sub
        
        print("Best C:", best_C)
        print("Best gamma:", best_g)


    return best_svm, y_sub, p_sub, y_hat, p_hat, best_C, best_g

### MATSVM


In [ ]:
t = TicToc()
t.tic()

for i in range(nalgos):
    t_inner = TicToc()
    t_inner.tic()

    state = np.random.get_state()
    np.random.seed(0)  # equivalent to MATLAB's rng('default') ?

    # y_b = [row[i] for row in y]
    y_b = y[:, i]

    
    # REQUIRE: Test case for validation the result
    # shuffle = True means that it partitions the data as it is presented. If the data is already in a random 
    # order, this isn't typically an issue, but if the data has some sort of order (e.g., all instances of one 
    # class followed by all instances of another), stratification alone won't randomize the order of the instances within each fold.

    # skf = StratifiedKFold(n_splits = opts_csv_fold, shuffle = True, random_state = 0)
    # TODO Xin : test it (exact match is not required, but difference has to be minimise)
    skf = KFold(n_splits = opts_csv_fold, shuffle = True, random_state = 0)
    
    # Test k-fold cross validation
    # data_splits = skf.split(z_norm, y_b)
    # test_split = next(data_splits)
    # print("Train indices: ", test_split[0][0:100])
    # print("Test indices: ", test_split[1][0:100])

    # fit_matsvm(z, y_b, w[:, i], skf, kernel_fcn, np.nan)

    best_svm, y_sub, p_sub, y_hat, p_hat, best_C, best_g = fit_matsvm(z, y_b, w[:, i], skf, kernel_fcn, np.nan)
    aux = confusion_matrix(y_b, y_sub)

    # np.prod(aux.shape) != 4 is False
    cvcmat[i, :] = aux.flatten()
    models_left = nalgos - (i + 1)
    print(f"    -> PYTHIA has trained a model for {algo_labels[i]}, there are {models_left} models left to train.")

    print(f"      -> Elapsed time: {t_inner.tocvalue():.2f}s")

    

### Estimation

In [ ]:
  
tn, fp, fn, tp = cvcmat[:, 0], cvcmat[:, 1], cvcmat[:, 2], cvcmat[:, 3]
print(tn.dtype)  
print(tn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
accuracy = (tp + tn) / ninst

print(precision)
# print(recall)
# print(accuracy)

### LIBSVM

In [ ]:
y = pd.read_csv('./data/ybin.csv')
y = y.values.tolist()

z = pd.read_csv('./data/z.csv').values.tolist()
z_norm = zscore(z, axis = 0, ddof = 1)

for i in range(nalgos):
    t_inner = TicToc()
    t_inner.tic()

    state = np.random.get_state()
    np.random.seed(0)  # equivalent to MATLAB's rng('default') ?

    # REQUIRE: Test case for validation the result
    y_b = [row[i] for row in y]
    # y_b = y[:, i]
    skf = StratifiedKFold(n_splits = opts_csv_fold, shuffle = True, random_state = 0)
    
    kkv= dict()
    for i, (train_index, test_index) in enumerate(skf.split(np.zeros(len(y_b)), y_b)):
        kkv[i] = [train_index.tolist(), test_index.tolist()]
    
    # start training using svm
    svm_res = fit_libsvm(z_norm, y_b, kkv, kernel_fcn)

    # visualise accuracy score
for k, v in svm_res.items():
    print(f'{k} fold: accuracy score = {v}')